# Dataset formats and type

- ***Định dạng*** (`format`) của một `dataset` đề cập đến cách dữ liệu được cấu trúc, thường được phân loại là *standard* (tiêu chuẩn) hoặc *conversational* (hội thoại).
- ***Loại*** (`type`) được liên kết với tác vụ cụ thể mà `dataset` được thiết kế, chẳng hạn như *prompt-only* (chỉ có prompt) hoặc *preference* (sở thích). Mỗi loại được đặc trưng bởi các cột của nó, các cột này thay đổi tùy theo tác vụ, như được trình bày trong bảng.

In [1]:
# %%capture
# !pip install trl

#### Standard (Tiêu chuẩn)

Định dạng `dataset` tiêu chuẩn thường bao gồm các chuỗi văn bản thuần túy. Các cột trong `dataset` thay đổi tùy thuộc vào tác vụ. Đây là định dạng được các `trainer` của `TRL` mong đợi. Dưới đây là các ví dụ về định dạng `dataset` tiêu chuẩn cho các tác vụ khác nhau:

In [2]:
# Language modeling
language_modeling_example = {"text": "The sky is blue."}
# Preference
preference_example = {"prompt": "The sky is", "chosen": " blue.", "rejected": " green."}
# Unpaired preference
unpaired_preference_example = {"prompt": "The sky is", "completion": " blue.", "label": True}


#### Conversational (Hội thoại)

Các `dataset` hội thoại được sử dụng cho các tác vụ liên quan đến đối thoại hoặc tương tác trò chuyện giữa người dùng và trợ lý. Không giống như các định dạng `dataset` tiêu chuẩn, chúng chứa các chuỗi tin nhắn, trong đó mỗi tin nhắn có một `role` (vai trò, ví dụ: `"user"` hoặc `"assistant"`) và `content` (nội dung, tức văn bản tin nhắn).

In [3]:
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
    {"role": "user", "content": "I'd like to show off how chat templating works!"},
]


Cũng giống như các `dataset` tiêu chuẩn, các cột trong `dataset` hội thoại thay đổi tùy thuộc vào tác vụ. Dưới đây là các ví dụ về định dạng `dataset` hội thoại cho các tác vụ khác nhau:

In [4]:
# Prompt-completion
prompt_completion_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}],
                             "completion": [{"role": "assistant", "content": "It is blue."}]}
# Preference
preference_example = {
    "prompt": [{"role": "user", "content": "What color is the sky?"}],
    "chosen": [{"role": "assistant", "content": "It is blue."}],
    "rejected": [{"role": "assistant", "content": "It is green."}],
}

Các `dataset` hội thoại rất hữu ích để huấn luyện các mô hình trò chuyện (`chat models`), nhưng phải được chuyển đổi sang định dạng `standard` trước khi sử dụng với các `trainer` của `TRL`. Điều này thường được thực hiện bằng cách sử dụng các `chat templates` dành riêng cho mô hình đang được sử dụng. Để biết thêm thông tin, hãy tham khảo phần [Làm việc với dataset hội thoại trong TRL](https://www.google.com/search?q=%23l%C3%A0m-vi%E1%BB%87c-v%E1%BB%9Bi-dataset-h%E1%BB%99i-tho%E1%BA%A1i-trong-trl).

## Tool Calling (Gọi công cụ)

Một số `chat templates` hỗ trợ *tool calling*, cho phép mô hình tương tác với các hàm bên ngoài—được gọi là **tools** (công cụ)—trong quá trình sinh văn bản. Điều này mở rộng khả năng hội thoại của mô hình bằng cách cho phép nó xuất ra một trường `"tool_calls"` thay vì một tin nhắn `"content"` tiêu chuẩn mỗi khi nó quyết định gọi một công cụ.

Sau khi trợ lý khởi tạo một lệnh gọi công cụ, công cụ sẽ thực thi và trả về kết quả. Trợ lý sau đó có thể xử lý kết quả này và tiếp tục cuộc hội thoại một cách phù hợp.

Đây là một ví dụ đơn giản về tương tác gọi công cụ:

In [5]:
messages = [
    {"role": "user", "content": "Turn on the living room lights."},
    {"role": "assistant", "tool_calls": [
        {"type": "function", "function": {
            "name": "control_light",
            "arguments": {"room": "living room", "state": "on"}
        }}]
    },
    {"role": "tool", "name": "control_light", "content": "The lights in the living room are now on."},
    {"role": "assistant", "content": "Done!"}
]

Khi chuẩn bị `dataset` cho Supervised Fine-Tuning (`SFT`) với `tool calling`, điều quan trọng là `dataset` của bạn phải bao gồm một cột bổ sung có tên là `tools`. Cột này chứa danh sách các công cụ có sẵn cho mô hình, thường được `chat template` sử dụng để xây dựng `system prompt`.

Các công cụ phải được chỉ định ở định dạng `JSON schema` đã được mã hóa. Bạn có thể tự động tạo `schema` này từ chữ ký hàm Python bằng cách sử dụng tiện ích [`~transformers.utils.get_json_schema`]:

In [6]:
from transformers.utils import get_json_schema

def control_light(room: str, state: str) -> str:
    """
    Controls the lights in a room.

    Args:
        room: The name of the room.
        state: The desired state of the light ("on" or "off").

    Returns:
        str: A message indicating the new state of the lights.
    """
    return f"The lights in {room} are now {state}."

# Generate JSON schema
json_schema = get_json_schema(control_light)
json_schema

/home/dino/Documents/learn-llm/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'type': 'function',
 'function': {'name': 'control_light',
  'description': 'Controls the lights in a room.',
  'parameters': {'type': 'object',
   'properties': {'room': {'type': 'string',
     'description': 'The name of the room.'},
    'state': {'type': 'string',
     'description': 'The desired state of the light ("on" or "off").'}},
   'required': ['room', 'state']},
  'return': {'type': 'string',
   'description': 'str: A message indicating the new state of the lights.'}}}

A complete dataset entry for SFT might look like:

In [7]:
{"messages": messages, "tools": [json_schema]}

{'messages': [{'role': 'user', 'content': 'Turn on the living room lights.'},
  {'role': 'assistant',
   'tool_calls': [{'type': 'function',
     'function': {'name': 'control_light',
      'arguments': {'room': 'living room', 'state': 'on'}}}]},
  {'role': 'tool',
   'name': 'control_light',
   'content': 'The lights in the living room are now on.'},
  {'role': 'assistant', 'content': 'Done!'}],
 'tools': [{'type': 'function',
   'function': {'name': 'control_light',
    'description': 'Controls the lights in a room.',
    'parameters': {'type': 'object',
     'properties': {'room': {'type': 'string',
       'description': 'The name of the room.'},
      'state': {'type': 'string',
       'description': 'The desired state of the light ("on" or "off").'}},
     'required': ['room', 'state']},
    'return': {'type': 'string',
     'description': 'str: A message indicating the new state of the lights.'}}}]}

### Types

#### Mô hình hóa ngôn ngữ (Language modeling)

Một `dataset` mô hình hóa ngôn ngữ bao gồm một cột `"text"` (hoặc `"messages"` cho các `dataset` hội thoại) chứa một chuỗi văn bản hoàn chỉnh.

In [8]:
# Standard format
language_modeling_example = {"text": "The sky is blue."}
# Conversational format
language_modeling_example = {"messages": [
    {"role": "user", "content": "What color is the sky?"},
    {"role": "assistant", "content": "It is blue."}
]}

#### Chỉ có prompt (Prompt-only)

Trong một `dataset` chỉ có `prompt`, chỉ có `prompt` ban đầu (câu hỏi hoặc câu chưa hoàn chỉnh) được cung cấp dưới khóa `"prompt"`. Quá trình huấn luyện thường bao gồm việc tạo ra `completion` dựa trên `prompt` này, nơi mô hình học cách tiếp tục hoặc hoàn thành đầu vào đã cho.

In [9]:
# Standard format
prompt_only_example = {"prompt": "The sky is"}
# Conversational format
prompt_only_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}]}

> Mặc dù cả hai loại `prompt-only` và `language modeling` đều tương tự nhau, chúng khác nhau ở cách xử lý đầu vào. Trong loại `prompt-only`, `prompt` đại diện cho một đầu vào chưa hoàn chỉnh mà mong đợi mô hình sẽ hoàn thành hoặc tiếp tục, trong khi ở loại `language modeling`, đầu vào được coi là một câu hoặc chuỗi hoàn chỉnh. Hai loại này được `TRL` xử lý khác nhau. Dưới đây là một ví dụ cho thấy sự khác biệt trong đầu ra của hàm `apply_chat_template` cho mỗi loại:

In [ ]:
from transformers import AutoTokenizer
from trl import apply_chat_template

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3n-E4B")

# Example for prompt-only type
prompt_only_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}]}
apply_chat_template(prompt_only_example, tokenizer)
# Output: {'prompt': '<|user|>\nWhat color is the sky?<|end|>\n<|assistant|>\n'}

# Example for language modeling type
lm_example = {"messages": [{"role": "user", "content": "What color is the sky?"}]}
apply_chat_template(lm_example, tokenizer)
# Output: {'text': '<|user|>\nWhat color is the sky?<|end|>\n<|endoftext|>'}

{"timestamp":"2025-07-22T09:44:22.951030Z","level":"WARN","fields":{"message":"Reqwest(reqwest::Error { kind: Request, source: hyper_util::client::legacy::Error(Connect, Custom { kind: Other, error: Os { code: 110, kind: TimedOut, message: \"Connection timed out\" } }) }). Retrying..."},"filename":"/home/runner/work/xet-core/xet-core/cas_client/src/http_client.rs","line_number":242}
{"timestamp":"2025-07-22T09:44:22.951086Z","level":"WARN","fields":{"message":"Retry attempt #0. Sleeping 2.319349394s before the next attempt"},"filename":"/root/.cargo/registry/src/index.crates.io-1949cf8c6b5b557f/reqwest-retry-0.7.0/src/middleware.rs","line_number":171}


- Đầu ra của `prompt-only` bao gồm một `'<|assistant|>\n'`, cho biết sự bắt đầu lượt của trợ lý và mong đợi mô hình tạo ra một `completion`.
- Ngược lại, đầu ra của `language modeling` coi đầu vào là một chuỗi hoàn chỉnh và kết thúc nó bằng `'<|endoftext|>'`, báo hiệu sự kết thúc của văn bản và không mong đợi bất kỳ nội dung bổ sung nào.

#### Prompt và completion (Prompt-completion)

Một `dataset` `prompt-completion` bao gồm một `"prompt"` và một `"completion"`.

In [ ]:
# Standard format
prompt_completion_example = {"prompt": "The sky is", "completion": " blue."}
# Conversational format
prompt_completion_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}],
                            "completion": [{"role": "assistant", "content": "It is blue."}]}

#### Preference

Một `dataset` sở thích (`preference`) được sử dụng cho các tác vụ mà mô hình được huấn luyện để chọn giữa hai hoặc nhiều `completion` có thể có cho cùng một `prompt`. `Dataset` này bao gồm một `"prompt"`, một `completion` `"chosen"` (được chọn), và một `completion` `"rejected"` (bị từ chối). Mô hình được huấn luyện để chọn câu trả lời `"chosen"` thay vì câu trả lời `"rejected"`.
Một số `dataset` có thể không bao gồm cột `"prompt"`, trong trường hợp đó `prompt` là ngầm định và được bao gồm trực tiếp trong các `completion` `"chosen"` và `"rejected"`. Chúng tôi khuyên bạn nên sử dụng `prompt` tường minh bất cứ khi nào có thể.


In [ ]:
# Standard format
## Explicit prompt (recommended)
preference_example = {"prompt": "The sky is", "chosen": " blue.", "rejected": " green."}
# Implicit prompt
preference_example = {"chosen": "The sky is blue.", "rejected": "The sky is green."}

# Conversational format
## Explicit prompt (recommended)
preference_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}],
                    "chosen": [{"role": "assistant", "content": "It is blue."}],
                    "rejected": [{"role": "assistant", "content": "It is green."}]}
## Implicit prompt
preference_example = {"chosen": [{"role": "user", "content": "What color is the sky?"},
                                {"role": "assistant", "content": "It is blue."}],
                    "rejected": [{"role": "user", "content": "What color is the sky?"},
                                {"role": "assistant", "content": "It is green."}]}

Để xem các ví dụ về `dataset` sở thích, hãy tham khảo [bộ sưu tập Preference datasets](https://huggingface.co/collections/trl-lib/preference-datasets-677e99b581018fcad9abd82c).

Một số `dataset` sở thích có thể được tìm thấy với [thẻ `dpo` trên Hugging Face Hub](https://www.google.com/search?q=%5Bhttps://huggingface.co/datasets%3Fother%3Ddpo%5D\(https://huggingface.co/datasets%3Fother%3Ddpo\)). Bạn cũng có thể khám phá [DPO Collections của librarian-bots](https://huggingface.co/collections/librarian-bots/direct-preference-optimization-datasets-66964b12835f46289b6ef2fc) để xác định các `dataset` sở thích.

#### Sở thích không theo cặp (Unpaired preference)

Một `dataset` sở thích không theo cặp (`unpaired preference`) tương tự như một `dataset` sở thích nhưng thay vì có các `completion` `"chosen"` và `"rejected"` cho cùng một `prompt`, nó bao gồm một `"completion"` duy nhất và một `"label"` cho biết `completion` đó có được ưa thích hay không.


In [ ]:
# Standard format
unpaired_preference_example = {"prompt": "The sky is", "completion": " blue.", "label": True}
# Conversational format
unpaired_preference_example = {"prompt": [{"role": "user", "content": "What color is the sky?"}],
                               "completion": [{"role": "assistant", "content": "It is blue."}],
                               "label": True}

Để xem các ví dụ về `dataset` sở thích không theo cặp, hãy tham khảo [bộ sưu tập Unpaired preference datasets](https://huggingface.co/collections/trl-lib/unpaired-preference-datasets-677ea22bf5f528c125b0bcdf).

#### Giám sát theo từng bước (Stepwise supervision)

Một `dataset` giám sát theo từng bước (hoặc quy trình) tương tự như một `dataset` [sở thích không theo cặp](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-kh%C3%B4ng-theo-c%E1%BA%B7p-unpaired-preference) nhưng bao gồm nhiều bước của `completion`, mỗi bước có nhãn riêng. Cấu trúc này hữu ích cho các tác vụ cần ghi nhãn chi tiết, từng bước, chẳng hạn như các tác vụ suy luận. Bằng cách đánh giá từng bước riêng biệt và cung cấp các nhãn mục tiêu, phương pháp này giúp xác định chính xác nơi suy luận đúng và nơi xảy ra lỗi, cho phép phản hồi có mục tiêu trên từng phần của quá trình suy luận.

In [ ]:
stepwise_example = {
    "prompt": "Which number is larger, 9.8 or 9.11?",
    "completions": ["The fractional part of 9.8 is 0.8, while the fractional part of 9.11 is 0.11.", "Since 0.11 is greater than 0.8, the number 9.11 is larger than 9.8."],
    "labels": [True, False]
}

Để xem các ví dụ về `dataset` giám sát theo từng bước, hãy tham khảo [bộ sưu tập Stepwise supervision datasets](https://huggingface.co/collections/trl-lib/stepwise-supervision-datasets-677ea27fd4c5941beed7a96e).

## Nên sử dụng loại dataset nào?

Việc chọn đúng loại `dataset` phụ thuộc vào tác vụ bạn đang thực hiện và các yêu cầu cụ thể của `trainer` `TRL` bạn đang sử dụng. Dưới đây là tổng quan ngắn gọn về các loại `dataset` được hỗ trợ bởi mỗi `trainer` `TRL`.

| Trainer                 | Loại dataset mong đợi                                                                                  |
| ----------------------- | ------------------------------------------------------------------------------------------------------ |
| [`BCOTrainer`]          | [Sở thích không theo cặp](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-kh%C3%B4ng-theo-c%E1%BA%B7p-unpaired-preference)                                                            |
| [`CPOTrainer`]          | [Sở thích (khuyến nghị prompt tường minh)](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-preference)                                                |
| [`DPOTrainer`]          | [Sở thích (khuyến nghị prompt tường minh)](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-preference)                                                |
| [`GKDTrainer`]          | [Prompt và completion](https://www.google.com/search?q=%23prompt-v%C3%A0-completion-prompt-completion)                                                    |
| [`GRPOTrainer`]         | [Chỉ có prompt](https://www.google.com/search?q=%23ch%E1%BB%89-c%C3%B3-prompt-prompt-only)                                                                            |
| [`IterativeSFTTrainer`] | [Sở thích không theo cặp](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-kh%C3%B4ng-theo-c%E1%BA%B7p-unpaired-preference)                                                            |
| [`KTOTrainer`]          | [Sở thích không theo cặp](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-kh%C3%B4ng-theo-c%E1%BA%B7p-unpaired-preference) hoặc [Sở thích (khuyến nghị prompt tường minh)](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-preference) |
| [`NashMDTrainer`]       | [Chỉ có prompt](https://www.google.com/search?q=%23ch%E1%BB%89-c%C3%B3-prompt-prompt-only)                                                                            |
| [`OnlineDPOTrainer`]    | [Chỉ có prompt](https://www.google.com/search?q=%23ch%E1%BB%89-c%C3%B3-prompt-prompt-only)                                                                            |
| [`ORPOTrainer`]         | [Sở thích (khuyến nghị prompt tường minh)](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-preference)                                                |
| [`PPOTrainer`]          | Mô hình hóa ngôn ngữ đã được token hóa (`Tokenized language modeling`)                                                                            |
| [`PRMTrainer`]          | [Giám sát theo từng bước](https://www.google.com/search?q=%23gi%C3%A1m-s%C3%A1t-theo-t%E1%BB%ABng-b%C6%B0%E1%BB%9Bc-stepwise-supervision)                                                          |
| [`RewardTrainer`]       | [Sở thích (khuyến nghị prompt ngầm định)](https://www.google.com/search?q=%23s%E1%BB%9F-th%C3%ADch-preference)                                                |
| [`SFTTrainer`]          | [Mô hình hóa ngôn ngữ](https://www.google.com/search?q=%23m%C3%B4-h%C3%ACnh-h%C3%B3a-ng%C3%B4n-ng%E1%BB%AF-language-modeling) hoặc [Prompt và completion](https://www.google.com/search?q=%23prompt-v%C3%A0-completion-prompt-completion)                     |
| [`XPOTrainer`]          | [Chỉ có prompt](https://www.google.com/search?q=%23ch%E1%BB%89-c%C3%B3-prompt-prompt-only) 

> Các `trainer` của `TRL` chỉ hỗ trợ các định dạng `dataset` tiêu chuẩn, [tính đến thời điểm hiện tại](https://github.com/huggingface/trl/issues/2071). Nếu bạn có một `dataset` hội thoại, trước tiên bạn phải chuyển đổi nó sang định dạng tiêu chuẩn.
Để biết thêm thông tin về cách làm việc với `dataset` hội thoại, hãy tham khảo phần [Làm việc với dataset hội thoại trong TRL](https://www.google.com/search?q=%23l%C3%A0m-vi%E1%BB%87c-v%E1%BB%9Bi-dataset-h%E1%BB%99i-tho%E1%BA%A1i-trong-trl).

## Làm việc với dataset hội thoại trong TRL

Các `dataset` hội thoại ngày càng phổ biến, đặc biệt là để huấn luyện các mô hình trò chuyện (`chat models`). Tuy nhiên, một số `trainer` của `TRL` không hỗ trợ `dataset` hội thoại ở định dạng thô của chúng. (Để biết thêm thông tin, xem [issue \#2071](https://github.com/huggingface/trl/issues/2071).) Các `dataset` này trước tiên phải được chuyển đổi sang định dạng tiêu chuẩn.
May mắn thay, `TRL` cung cấp các công cụ để dễ dàng xử lý việc chuyển đổi này, được trình bày chi tiết dưới đây.

### Chuyển đổi một dataset hội thoại thành một dataset tiêu chuẩn

Để chuyển đổi một `dataset` hội thoại thành một `dataset` tiêu chuẩn, bạn cần *áp dụng một chat template* cho `dataset`. Một `chat template` là một cấu trúc được xác định trước thường bao gồm các trình giữ chỗ cho tin nhắn của người dùng và trợ lý. `Template` này được cung cấp bởi `tokenizer` của mô hình bạn sử dụng.

Để biết hướng dẫn chi tiết về cách sử dụng `chat templating`, hãy tham khảo [mục Chat templating trong tài liệu của `transformers`](https://www.google.com/search?q=%5Bhttps://huggingface.co/docs/transformers/en/chat_templating%5D\(https://huggingface.co/docs/transformers/en/chat_templating\)).

Trong `TRL`, phương thức bạn áp dụng để chuyển đổi `dataset` sẽ thay đổi tùy thuộc vào tác vụ. May mắn thay, `TRL` cung cấp một hàm trợ giúp có tên là [`apply_chat_template`] để đơn giản hóa quá trình này. Đây là một ví dụ về cách sử dụng nó: